**Movie Recommendation System**

Step 1: Importing Required Library 

In [11]:
#Importing Required Library
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

Step 2: Loading Data Movie Credits and Movies

In [12]:
tmdb_credits=pd.read_csv("tmdb_5000_credits.csv")
tmdb_movies=pd.read_csv("tmdb_5000_movies.csv")

In [13]:
tmdb_movies.shape

(4803, 20)

In [14]:
tmdb_credits.shape

(4803, 4)

From Above shape it is clear that both have same shape and have one common column so merging dataframe on that column

In [15]:
df=pd.merge(tmdb_movies,tmdb_credits,on='title')

In [16]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [17]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

Now From Above We can See that Genre,Keywords,production_companies,cast and Crew, which we require for Our Recommendation System are not Properly arranged, so for that we have to remove unwanted words from them for that we will follow below Procedure

In [18]:
df.genres[1]#By this we can see that our genres contains dictionary so by calling name we will get Name

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

For Getting Name We are Making Function

In [19]:
def get_name(a):
    k=[]
    for i in ast.literal_eval(a):# As we can see that our data is str for that we have to use this func
        k.append(i["name"])
    return k

Simillarly Apply Function to Keywords,Production Companies

In [20]:
df['genres']=[get_name(sent) for sent in df.genres]
df['keywords']=[get_name(sent) for sent in df.keywords]
df['production_companies']=[get_name(sent) for sent in df.production_companies]

For Crew Feature We are Selecting Top Five Cast 

In [21]:
def get_name_5(a):
    k=[]
    counter=0
    for i in ast.literal_eval(a):
        if counter !=5:
            k.append(i["name"])
            counter+=1
        else:
            break        
    return k

In [22]:
df['cast']=[get_name_5(sent) for sent in df.cast]

In Crew We have many names but we are interested only in Director so fetching only for director

In [23]:
def fetch_director(sub):
    k=[]
    for i in ast.literal_eval(sub):
        if i['job']=='Director':
            k.append(i['name'])
            break
    return k

In [24]:
df['crew']=[fetch_director(sent) for sent in df.crew]

Now Displying Our Clean Data

In [25]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


Now In Some Cases We have Two Words like Science Fiction in this case system does not know they are single so the 
take it as different entity.so Replcing blank spaces between words

In [26]:
df['genres']=df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
df['keywords']=df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
df['production_companies']=df['production_companies'].apply(lambda x:[i.replace(" ","") for i in x])
df['cast']=df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
df['crew']=df['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [27]:
df['overview']=df['overview'].apply(lambda x:str(x).split()) #Spliting Overview for better adding

Now We are Only Interested In Content Base Model so for that we have to add data of Genre,Keyword,Overview,production company,Cast,Crew 
in Single Column so it will become easy to use bag of words

In [28]:
df['text']=df['genres']+df['keywords']+df['overview']+df['production_companies']+df['cast']+df['crew'];

In [29]:
df_new=df[['movie_id','title','text']]

In [30]:
df_new.head(1)

,movie_id,title,text
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."


In [31]:
df_new['text']=df_new['text'].apply(lambda x:" ".join(x))  # Now After Combining joining the data

<ipython-input-31-6929d7151ee7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['text']=df_new['text'].apply(lambda x:" ".join(x))  # Now After Combining joining the data


In [32]:
df_new['text']=df_new['text'].apply(lambda x:x.lower())
df_new.head()

<ipython-input-32-d02741dce550>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['text']=df_new['text'].apply(lambda x:x.lower())


,movie_id,title,text
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


Now Importing Porter Stemmer For Stemming of Words 

In [33]:
from nltk.stem import PorterStemmer

In [34]:
ps=PorterStemmer()

In [35]:
ps.stem('going')

'go'

In [36]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)
        

In [37]:
df_new['text']=df_new['text'].apply(stem)

<ipython-input-37-7d21ea19b90e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['text']=df_new['text'].apply(stem)


In [38]:
df_new['text'][1]

"adventur fantasi action ocean drugabus exoticisland eastindiatradingcompani loveofone'slif traitor shipwreck strongwoman ship allianc calypso afterlif fighter pirat swashbuckl aftercreditssting captain barbossa, long believ to be dead, ha come back to life and is head to the edg of the earth with will turner and elizabeth swann. but noth is quit as it seems. waltdisneypictur jerrybruckheimerfilm secondmateproduct johnnydepp orlandobloom keiraknightley stellanskarsgård chowyun-fat goreverbinski"

Importing Count Vectoriser for Conversion of Word to Vector

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
clf=CountVectorizer(max_features=5000,stop_words='english')

In [40]:
vectors=clf.fit_transform(df_new['text']).toarray()

In [41]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

**Cosine Similarity**

Recommendation system is based on Cosine Similarity
Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space.
It is defined to equal the cosine of the angle between them,
which is also the same as the inner product of the same vectors normalized to both have length 1

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
similarity=cosine_similarity(vectors)

Making Our Recommedation Model

In [44]:
def recommend(movie):
    movie_index=df_new[df_new['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6] #Getting 5 Recommendation
    
    for i in movies_list:
        print(df_new.iloc[i[0]].title)        

In [45]:
recommend('Superman')

Superman II
Superman Returns
Superman IV: The Quest for Peace
Superman III
Iron Man 2


Thanks